<a href="https://colab.research.google.com/github/markriis/IDS-pollution-of-cities/blob/combining/Data_prep.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Importing data and librarys

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd


In [27]:
gasses = pd.read_csv('https://raw.githubusercontent.com/owid/co2-data/master/owid-co2-data.csv',
                     usecols=['country', 'year', 'population', 'gdp', 'cement_co2',
       'cement_co2_per_capita', 'co2', 'co2_including_luc', 'co2_including_luc_growth_abs',
       'co2_including_luc_growth_prct', 'co2_including_luc_per_capita',
       'co2_including_luc_per_gdp', 'co2_including_luc_per_unit_energy',
       'co2_per_capita', 'co2_per_gdp', 'co2_per_unit_energy', 'coal_co2',
       'coal_co2_per_capita', 'cumulative_cement_co2', 'cumulative_co2',
       'cumulative_co2_including_luc', 'cumulative_coal_co2',
       'cumulative_flaring_co2', 'cumulative_gas_co2', 'cumulative_luc_co2',
       'cumulative_oil_co2', 'energy_per_capita',
       'energy_per_gdp', 'flaring_co2', 'flaring_co2_per_capita', 'gas_co2',
       'gas_co2_per_capita', 'ghg_excluding_lucf_per_capita', 'ghg_per_capita',
       'land_use_change_co2', 'land_use_change_co2_per_capita', 'methane',
       'methane_per_capita', 'nitrous_oxide', 'nitrous_oxide_per_capita',
       'oil_co2', 'oil_co2_per_capita', 'primary_energy_consumption',
       'total_ghg', 'total_ghg_excluding_lucf'])# Removed some columns that I doubt would be of use, meanings at https://github.com/owid/co2-data/blob/master/owid-co2-codebook.csv

air = pd.read_excel('https://cdn.who.int/media/docs/default-source/air-pollution-documents/air-quality-and-health/who_aap_2021_v9_11august2022.xlsx?sfvrsn=9035996c_3',
                    storage_options={'User-Agent': 'Mozilla/5.0'}, sheet_name='AAP_2022_city_v9',
                    usecols=['WHO Country Name', 'City or Locality', 'Measurement Year', 'PM2.5 (μg/m3)', 'PM10 (μg/m3)', 'NO2 (μg/m3)'])

hdr_cols = ['country']
for y in range(1995, 2022):
  hdr_cols.extend([f'hdi_{y}', f'le_{y}', f'gnipc_{y}'])

development = pd.read_csv('https://hdr.undp.org/sites/default/files/2021-22_HDR/HDR21-22_Composite_indices_complete_time_series.csv',
                          storage_options={'User-Agent': 'Mozilla/5.0'},
                          usecols=hdr_cols)#


populations = pd.read_csv('https://github.com/markriis/IDS-pollution-of-cities/raw/main/UNdata_Populations.zip',
                          usecols=['Country or Area', 'Year', 'City', 'Reliability', 'Value'],
                          nrows=27947)


Looking at datasets

In [28]:
air.head()

,WHO Country Name,City or Locality,Measurement Year,PM2.5 (μg/m3),PM10 (μg/m3),NO2 (μg/m3)
0,Afghanistan,Kabul,2019,119.77,NaN,NaN
1,Albania,Durres,2015,NaN,17.65,26.63
2,Albania,Durres,2016,14.32,24.56,24.78
3,Albania,Elbasan,2015,NaN,NaN,23.96
4,Albania,Elbasan,2016,NaN,NaN,26.26


In [29]:
development.head()

,country,hdi_1995,hdi_1996,hdi_1997,hdi_1998,hdi_1999,hdi_2000,hdi_2001,hdi_2002,hdi_2003,...,gnipc_2012,gnipc_2013,gnipc_2014,gnipc_2015,gnipc_2016,gnipc_2017,gnipc_2018,gnipc_2019,gnipc_2020,gnipc_2021
0,Afghanistan,0.310,0.319,0.323,0.324,0.332,0.335,0.337,0.362,0.376,...,2125.862821,2193.553936,2178.507021,2101.589319,2077.566899,2085.487571,2054.939895,2097.889450,1997.852149,1824.190915
1,Angola,NaN,NaN,NaN,NaN,0.364,0.375,0.386,0.403,0.420,...,7280.845666,7478.104777,7704.231949,7652.656486,7189.426672,6861.575738,6381.521946,6082.746624,5593.142060,5465.617791
2,Albania,0.634,0.645,0.642,0.657,0.669,0.677,0.684,0.689,0.696,...,11146.263030,11552.982470,11691.648290,12016.297600,12484.624200,12802.148310,13302.705960,13485.311240,12996.762910,14131.110390
3,Andorra,NaN,NaN,NaN,NaN,NaN,0.818,0.825,0.832,0.841,...,47126.814610,46385.095200,48483.720320,49936.874540,52267.738320,52650.225760,53483.306630,54465.047400,47878.666640,51166.626610
4,United Arab Emirates,0.762,0.767,0.773,0.779,0.787,0.796,0.800,0.804,0.814,...,57445.954750,60005.695360,62573.505310,65577.512240,66881.329740,67667.508460,67195.095230,68590.900940,63016.401220,62573.591810


In [30]:
gasses.head()

,country,year,population,gdp,cement_co2,cement_co2_per_capita,co2,co2_including_luc,co2_including_luc_growth_abs,co2_including_luc_growth_prct,...,land_use_change_co2_per_capita,methane,methane_per_capita,nitrous_oxide,nitrous_oxide_per_capita,oil_co2,oil_co2_per_capita,primary_energy_consumption,total_ghg,total_ghg_excluding_lucf
0,Afghanistan,1850,3752993.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.794,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Afghanistan,1851,3767956.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.797,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Afghanistan,1852,3783940.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.799,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Afghanistan,1853,3800954.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.799,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Afghanistan,1854,3818038.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.800,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [31]:
gasses = gasses[gasses['year'] >= 1995]
gasses.shape

(7240, 45)

In [32]:
populations.head()# I have no idea why some names are fully in caps

,Country or Area,Year,City,Reliability,Value
0,Åland Islands,2022,MARIEHAMN,"Final figure, complete",11750.0
1,Åland Islands,2021,MARIEHAMN,"Final figure, complete",11724.0
2,Åland Islands,2020,MARIEHAMN,"Final figure, complete",11692.0
3,Åland Islands,2019,MARIEHAMN,"Final figure, complete",11711.0
4,Åland Islands,2018,MARIEHAMN,"Final figure, complete",11709.0


In [33]:
populations['Value'] = populations['Value'].astype(int)# Number of people should be whole

populations = populations[populations['Reliability'] == 'Final figure, complete'].drop(columns = ['Reliability'])# Most of the data is final figure, I think we can skip the rest
populations = populations[populations['Year'] >= 1990]
populations.reset_index(inplace=True, drop=True)
populations.head()

,Country or Area,Year,City,Value
0,Åland Islands,2022,MARIEHAMN,11750
1,Åland Islands,2021,MARIEHAMN,11724
2,Åland Islands,2020,MARIEHAMN,11692
3,Åland Islands,2019,MARIEHAMN,11711
4,Åland Islands,2018,MARIEHAMN,11709


Finding what countries are in all dataframes

In [34]:
countries_air = air['WHO Country Name'].dropna().values
countries_dev = development['country'].dropna().values
countries_gas = gasses['country'].dropna().values
countries_pop = populations['Country or Area'].dropna().values

display(pd.unique(countries_air).shape)# It apears air has the least countries in it, but still 118 is prety good
display(pd.unique(countries_dev).shape)
display(pd.unique(countries_gas).shape)
display(pd.unique(countries_pop).shape)

(118,)

(206,)

(259,)

(209,)

In [35]:
common_countries = np.intersect1d(np.intersect1d(countries_air, countries_dev), np.intersect1d(countries_gas, countries_pop))
display(common_countries)
display(common_countries.shape)

array(['Albania', 'Algeria', 'Andorra', 'Argentina', 'Australia',
       'Austria', 'Bahamas', 'Bahrain', 'Bangladesh', 'Belgium', 'Bhutan',
       'Bosnia and Herzegovina', 'Brazil', 'Bulgaria', 'Cameroon',
       'Canada', 'Chile', 'China', 'Colombia', 'Costa Rica', 'Croatia',
       'Cuba', 'Czechia', 'Denmark', 'Ecuador', 'Egypt', 'El Salvador',
       'Estonia', 'Ethiopia', 'Fiji', 'Finland', 'France', 'Georgia',
       'Germany', 'Ghana', 'Greece', 'Guatemala', 'Honduras', 'Hungary',
       'Iceland', 'India', 'Indonesia', 'Ireland', 'Israel', 'Italy',
       'Jamaica', 'Japan', 'Jordan', 'Kazakhstan', 'Kenya', 'Kuwait',
       'Kyrgyzstan', 'Latvia', 'Lebanon', 'Liechtenstein', 'Lithuania',
       'Luxembourg', 'Madagascar', 'Malaysia', 'Maldives', 'Malta',
       'Mauritius', 'Mexico', 'Monaco', 'Mongolia', 'Montenegro',
       'Myanmar', 'Nepal', 'Netherlands', 'New Zealand', 'Nigeria',
       'North Macedonia', 'Norway', 'Pakistan', 'Panama', 'Paraguay',
       'Peru', 'Phili

(102,)

In [36]:
air.rename(columns={'WHO Country Name' : 'country', 'City or Locality' : 'city'}, inplace=True)
populations.rename(columns={'Country or Area' : 'country', 'City' : 'city'}, inplace=True)

datasets = {
    'gasses' : gasses,
    'air' : air,
    'development' : development,
    'populations' : populations
}

In [37]:
for name, dataset in datasets.items():
  datasets[name] = dataset[dataset['country'].isin(common_countries)]
  datasets[name].reset_index(inplace=True, drop=True)

In [38]:
for name, dataset in datasets.items():
  display(dataset.shape)

(2856, 45)

(28499, 6)

(102, 82)

(19256, 4)

Finding the cities that are in both air and population dataframes

In [39]:
air_country_cities = datasets['air']['country'] + '_' + datasets['air']['city']# Compareing city-country pairs encase there are several citys with same name
pop_country_cities = datasets['populations']['country'] + '_' + datasets['populations']['city']


In [40]:
def sName(df):# A function to get rid of diferences in the names that are most likely to ocurr, because of human error
  return df.str.lower().str.replace(' |-', '', regex=True)

In [41]:
common_idx = np.intersect1d(sName(air_country_cities), sName(pop_country_cities), return_indices=True)[1]
common_idx

array([   19,    25,    54, ..., 28348, 28350, 28352])

In [42]:
common_cities = datasets['air']['city'].iloc[common_idx]

datasets['air'] = datasets['air'][datasets['air']['city'].isin(common_cities)] #
datasets['air'].reset_index(inplace=True, drop=True)

datasets['populations'] = datasets['populations'][sName(datasets['populations']['city']).isin(sName(common_cities))]
datasets['populations'].reset_index(inplace=True, drop=True)


In [43]:
for name, dataset in datasets.items():
  display(dataset.shape)

(2856, 45)

(6136, 6)

(102, 82)

(7948, 4)

Getting the city names in two datasets to match

In [44]:
datasets['populations'].loc[~datasets['populations']['city'].isin(common_cities), 'city'] = datasets['populations'].loc[~datasets['populations']['city'].isin(common_cities), 'city'].str.title()# Get rid of comleet uppercase

In [45]:
datasets['populations'][~datasets['populations']['city'].isin(common_cities)]['city'].unique()

array(['Guangan', 'Huaian', 'Linan', 'Weinan', 'Hubli-Dharwad',
       'Pimpri Chinchwad', 'Rae Bareli', 'Thiruvananthapuram',
       "Reggio Nell'Emilia", 'Vacoas - Phoenix', 'Bielsko-Biala',
       'Satu-Mare', 'Drobeta Turnu-Severin', "L'Hospitalet De Llobregat",
       'Port-Of-Spain'], dtype=object)

In [46]:
for city in datasets['populations'][~datasets['populations']['city'].isin(common_cities)]['city'].unique(): #Iterate through the city names that still dont match

  location = sName(common_cities)[sName(common_cities) == (city.lower().replace('-', '').replace(' ', ''))].index[0]
  datasets['populations'].loc[datasets['populations']['city'] == city, 'city'] = common_cities[location]
  print(f'Replaced {city} with {str(common_cities[location])} from common_cities')


Replaced Guangan with Guang An from common_cities
Replaced Huaian with Huai An from common_cities
Replaced Linan with Lin An from common_cities
Replaced Weinan with Wei Nan from common_cities
Replaced Hubli-Dharwad with Hubli- Dharwad from common_cities
Replaced Pimpri Chinchwad with Pimpri- Chinchwad from common_cities
Replaced Rae Bareli with Raebareli from common_cities
Replaced Thiruvananthapuram with Thiruvanantha Puram from common_cities
Replaced Reggio Nell'Emilia with Reggio Nell'emilia from common_cities
Replaced Vacoas - Phoenix with Vacoas-Phoenix from common_cities
Replaced Bielsko-Biala with Bielsko Biala from common_cities
Replaced Satu-Mare with Satu Mare from common_cities
Replaced Drobeta Turnu-Severin with Drobeta Turnu Severin from common_cities
Replaced L'Hospitalet De Llobregat with L'hospitalet De Llobregat from common_cities
Replaced Port-Of-Spain with Port Of Spain from common_cities


In [47]:
datasets['populations'][~datasets['populations']['city'].isin(common_cities)].shape

(0, 4)

A look at the processed data

In [48]:
for name, dataset in datasets.items():
  display(dataset)

,country,year,population,gdp,cement_co2,cement_co2_per_capita,co2,co2_including_luc,co2_including_luc_growth_abs,co2_including_luc_growth_prct,...,land_use_change_co2_per_capita,methane,methane_per_capita,nitrous_oxide,nitrous_oxide_per_capita,oil_co2,oil_co2_per_capita,primary_energy_consumption,total_ghg,total_ghg_excluding_lucf
0,Albania,1995,3284370.0,1.160073e+10,0.103,0.031,2.089,2.581,-0.034,-1.309,...,0.150,4.08,1.242,1.35,0.411,1.773,0.540,22.728,7.59,7.40
1,Albania,1996,3271336.0,1.272738e+10,0.105,0.032,2.018,2.318,-0.264,-10.212,...,0.092,3.92,1.198,1.20,0.367,1.755,0.536,25.011,7.32,7.12
2,Albania,1997,3253721.0,1.139651e+10,0.052,0.016,1.543,1.788,-0.530,-22.865,...,0.075,3.72,1.143,1.14,0.350,1.411,0.434,22.844,6.59,6.37
3,Albania,1998,3232185.0,1.246671e+10,0.044,0.013,1.755,2.002,0.214,11.992,...,0.077,3.57,1.105,1.17,0.362,1.623,0.502,24.572,6.82,6.62
4,Albania,1999,3208265.0,1.415186e+10,0.055,0.017,2.986,2.956,0.954,47.637,...,-0.010,3.62,1.128,1.14,0.355,2.792,0.870,22.114,8.03,7.84
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2851,Uzbekistan,2018,32449578.0,3.368748e+11,3.776,0.116,102.288,100.672,-6.584,-6.139,...,-0.050,54.25,1.672,16.62,0.512,11.692,0.360,533.657,181.47,184.84
2852,Uzbekistan,2019,32976946.0,NaN,4.511,0.137,108.429,107.044,6.372,6.329,...,-0.042,54.44,1.651,16.78,0.509,11.314,0.343,534.630,186.84,190.21
2853,Uzbekistan,2020,33526662.0,NaN,5.147,0.154,109.632,108.379,1.335,1.247,...,-0.037,53.60,1.599,16.99,0.507,12.696,0.379,531.753,184.15,187.52
2854,Uzbekistan,2021,34081452.0,NaN,6.157,0.181,116.388,115.257,6.878,6.346,...,-0.033,NaN,NaN,NaN,NaN,14.700,0.431,564.146,NaN,NaN


,country,city,Measurement Year,PM2.5 (μg/m3),PM10 (μg/m3),NO2 (μg/m3)
0,Argentina,Buenos Aires,2014,10.11,NaN,NaN
1,Argentina,Buenos Aires,2015,10.26,27.87,NaN
2,Argentina,Buenos Aires,2016,NaN,26.88,15.35
3,Argentina,Buenos Aires,2017,NaN,25.40,19.57
4,Argentina,Buenos Aires,2018,NaN,24.40,16.60
...,...,...,...,...,...,...
6131,Ukraine,Kyiv,2019,23.36,31.19,NaN
6132,Uruguay,Montevideo,2014,8.00,25.83,NaN
6133,Uruguay,Montevideo,2018,12.00,19.00,NaN
6134,Uzbekistan,Tashkent,2018,43.09,NaN,NaN


,country,hdi_1995,hdi_1996,hdi_1997,hdi_1998,hdi_1999,hdi_2000,hdi_2001,hdi_2002,hdi_2003,...,gnipc_2012,gnipc_2013,gnipc_2014,gnipc_2015,gnipc_2016,gnipc_2017,gnipc_2018,gnipc_2019,gnipc_2020,gnipc_2021
0,Albania,0.634,0.645,0.642,0.657,0.669,0.677,0.684,0.689,0.696,...,11146.263030,11552.982470,11691.648290,12016.297600,12484.624200,12802.148310,13302.705960,13485.311240,12996.762910,14131.110390
1,Andorra,NaN,NaN,NaN,NaN,NaN,0.818,0.825,0.832,0.841,...,47126.814610,46385.095200,48483.720320,49936.874540,52267.738320,52650.225760,53483.306630,54465.047400,47878.666640,51166.626610
2,Argentina,0.745,0.751,0.756,0.762,0.773,0.779,0.784,0.785,0.793,...,23557.513140,23915.419720,23087.869640,23508.767380,22694.285100,22996.671420,21924.731190,21197.741040,19178.331840,20925.268140
3,Australia,0.882,0.884,0.887,0.891,0.894,0.896,0.900,0.902,0.906,...,44878.417640,45260.609170,45851.642010,46307.576170,46558.178270,47085.227140,47487.644070,47312.192000,47078.244460,49238.433350
4,Austria,0.849,0.853,0.857,0.863,0.866,0.871,0.876,0.879,0.881,...,52887.194300,52625.835170,52720.297690,52450.238770,53662.574170,53665.026680,54587.744450,55323.039150,51574.183060,53618.671380
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97,Uganda,0.342,0.354,0.364,0.373,0.385,0.394,0.404,0.417,0.429,...,1955.886259,1959.621180,1988.490664,2028.218794,2051.754604,2025.816431,2064.075583,2124.933537,2138.005821,2181.436510
98,Ukraine,0.684,0.685,0.689,0.696,0.698,0.700,0.715,0.722,0.732,...,12839.410110,12824.773420,12237.292450,11671.553210,11649.481050,12032.754730,12455.669450,12964.436140,12718.460510,13255.509770
99,Uruguay,0.717,0.723,0.735,0.744,0.747,0.753,0.760,0.764,0.770,...,19381.352120,20666.306670,21004.102730,21439.388020,21673.660350,21733.372050,21730.725670,21916.768220,20441.929130,21268.881220
100,Uzbekistan,NaN,NaN,NaN,NaN,NaN,0.607,0.614,0.621,0.627,...,5599.849102,5977.987646,6258.947949,6485.233243,6725.763449,6974.358065,7303.472178,7499.478344,7512.022906,7916.785725


,country,Year,city,Value
0,Argentina,2022,Buenos Aires,15716718
1,Argentina,2021,Buenos Aires,15567820
2,Argentina,2020,Buenos Aires,15416728
3,Argentina,2019,Buenos Aires,15263440
4,Argentina,2016,Buenos Aires,13879707
...,...,...,...,...
7943,Uzbekistan,2018,Tashkent,2464933
7944,Uzbekistan,2017,Tashkent,2424146
7945,Uzbekistan,2016,Tashkent,2393176
7946,Uzbekistan,2015,Tashkent,2371269


In [49]:
g = datasets['populations'].groupby(['country', 'city'])

g.min()

Year    Value
country    city                        
Argentina  Buenos Aires   1991  2965403
Australia  Adelaide       2001    24193
           Brisbane       2001  1209322
           Canberra       2001   308764
           Central Coast  2001   254579
...                        ...      ...
Tunisia    Tunis          2014  1056247
Uganda     Kampala        2002  1208544
Ukraine    Kyiv           2001  2566953
Uruguay    Montevideo     2003  1305082
Uzbekistan Tashkent       2001  2137218

[1103 rows x 2 columns]

In [50]:
datasets['development'].isna().sum()

country        0
hdi_1995      13
hdi_1996      13
hdi_1997      13
hdi_1998      13
              ..
gnipc_2017     1
gnipc_2018     1
gnipc_2019     1
gnipc_2020     1
gnipc_2021     1
Length: 82, dtype: int64

In [51]:
datasets['populations']['Year'].value_counts()

2011    794
2010    537
2001    530
2016    438
2006    424
2020    403
2000    393
2009    363
2015    358
2018    339
2021    321
2017    319
2007    311
2008    297
2012    281
2003    254
2013    252
2005    251
2014    242
2019    220
2004    180
1999    146
2022    137
2002     77
2023     52
1991     13
1998      7
1995      5
1993      2
1992      1
1996      1
Name: Year, dtype: int64

In [52]:
import pickle

pickle.dump(datasets, open( "datasets.p", "wb" ) )